In [1]:
# !pip install captcha -i https://pypi.douban.com/simple

In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
import random
import time
import datetime
from captcha.image import ImageCaptcha
from PIL import Image,ImageFont,ImageDraw

# 定义一些常量
# 元数据集
DIGITS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# 图片大小
OUTPUT_SHAPE = (32, 256)

# 训练最大轮次
num_epochs = 50000
num_hidden = 128
num_layers = 2

#最多11个类别
num_classes = len(DIGITS) + 1 

# 初始化学习速率
INITIAL_LEARNING_RATE = 1e-3
DECAY_STEPS = 5000
REPORT_STEPS = 100
LEARNING_RATE_DECAY_FACTOR = 0.9  
MOMENTUM = 0.9

BATCHES = 10
BATCH_SIZE = 64
TRAIN_SIZE = BATCHES * BATCH_SIZE

In [2]:
# !pip install captcha -i https://pypi.douban.com/simple
    

In [3]:
# !mkdir  lstm_ctc_data lstm_ctc_model


In [4]:
data_dir = 'lstm_ctc_data/'
model_dir = '/lstm_ctc_model/'

In [5]:
#椒盐噪声
#就是把图像中的所有像素点取出一定的百分比，随机用黑和白色像素随机赋值
def img_salt_pepper_noise(src,percetage):
    NoiseImg=src
    NoiseNum=int(percetage*src.shape[0]*src.shape[1])
    for i in range(NoiseNum):
        randX=random.randint(0,src.shape[0]-1)
        randY=random.randint(0,src.shape[1]-1)
        if random.randint(0,1)==0:
            #0是黑色
            NoiseImg[randX,randY]=0
        else:
            #255是白色
            NoiseImg[randX,randY]=255
    return NoiseImg

In [6]:
# 随机生成不定长数据
def gen_text_2(cnt):
    font_path = 'arial.ttf'
    font_size = 30
    font=ImageFont.truetype(font_path,font_size)
 
    #创建一定数量的图片
    for i in range(cnt):
        rnd = random.randint(1, 10)
        text = ''
        for j in range(rnd):
            text = text + DIGITS[random.randint(0, len(DIGITS) - 1)]
            
        #新建空白图片
        img=Image.new("RGB",(256,32))
        #创建可操作对象
        draw=ImageDraw.Draw(img)
        #drawObject.text(position,  string,  options)Position是一个二元元组，指定字符串左上角坐标，string是要写入的字符串
        draw.text((1,1),text,font=font,fill='white')

        img=np.array(img)
        #对图像添加椒盐噪声，每个图像添加椒盐噪声的比例不同
        img = img_salt_pepper_noise(img, float(random.randint(1,10)/100.0))

        cv2.imwrite(data_dir + text + '_' + str(i+1) + '.jpg',img)


In [7]:
# #创建500张图片，运行程序时训练数据就是从这里读取的
# cnt=500
# gen_text_2(cnt)

In [8]:
# ImageFont.truetype('arial.ttf',30)

In [11]:
# 稀疏矩阵转序列
def decode_a_seq(indexes, spars_tensor):
    decoded = []
    for m in indexes:
        str = DIGITS[spars_tensor[1][m]]
        decoded.append(str)
    return decoded

def decode_sparse_tensor(sparse_tensor):
    decoded_indexes = list()
    current_i = 0
    current_seq = []

    for offset, i_and_index in enumerate(sparse_tensor[0]):
        i = i_and_index[0]
        if i != current_i:
            decoded_indexes.append(current_seq)
            current_i = i
            current_seq = list()
        current_seq.append(offset)
    decoded_indexes.append(current_seq)
    result = []
    for index in decoded_indexes:
        result.append(decode_a_seq(index, sparse_tensor))
    return result


In [12]:
# gen_text_2(5)

In [13]:
# 准确性评估
# 输入：预测结果序列 decoded_list ,目标序列 test_targets
# 返回：准确率
def report_accuracy(decoded_list, test_targets):
    original_list = decode_sparse_tensor(test_targets)
    detected_list = decode_sparse_tensor(decoded_list)

    # 正确数量
    true_numer = 0

    # 预测序列与目标序列的维度不一致，说明有些预测失败，直接返回
    if len(original_list) != len(detected_list):
        print("len(original_list)", len(original_list), "len(detected_list)", len(detected_list),
              " test and detect length desn't match")
        return

    # 比较预测序列与结果序列是否一致，并统计准确率        
    print("T/F: original(length) <-------> detectcted(length)")
    for idx, number in enumerate(original_list):
        detect_number = detected_list[idx]
        hit = (number == detect_number)
        print(hit, number, "(", len(number), ") <-------> ", detect_number, "(", len(detect_number), ")")
        if hit:
            true_numer = true_numer + 1
    accuracy = true_numer * 1.0 / len(original_list)
    print("Test Accuracy:", accuracy)    
    return accuracy

In [14]:
# 转化一个序列列表为稀疏矩阵
def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [15]:
# 将文件和标签读到数组
def get_file_text_array():
    file_name_array=[]
    text_array=[]

    for parent, dirnames, filenames in os.walk(data_dir):
        file_name_array=filenames

        for f in file_name_array:
            text = f.split('_')[0]
            text_array.append(text)

    return file_name_array,text_array
'''
file_name_array 1005 ['56_129.jpg', '2_162.jpg', '50268339_147.jpg', '490455_323.jpg', '32046500_2.jpg', '5966_212.jpg', '615643_397.jpg', '6_165.jpg', '881_378.jpg', '9093924_312.jpg', '4_497.jpg', '98085476_211.jpg', '8794_102.jpg', '051_377.jpg', '78_209.jpg', '9664420891_82.jpg', '732691_293.jpg',

text_array ['56', '2', '50268339', '490455', '32046500', '5966', '615643', '6', '881', '9093924', '4', '98085476', '8794', '051', '78', '9664420891', '732691', '32211', '01', '63', '00285323', '11', '8274', '4165161', '59', '1', '344', '673', '07966358', '1473599', '90540139', '5191676307', '82222', '2644860', '597084', '043212', '358754878', '7', '21', '109', '8', '9375727', '74942', '36614993', '715271419', '87', '1848415023', '111127145', '493261062', '4498', '94264', '403', '740939538', '37263', '655036489', '70', '699371688', '301167', '040294', '24', '011', '25960', '1', '784000633', '3177', '211', '9544', '3592', '378', '69814
'''

In [29]:
# help(cv2.resize)
# src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst

In [16]:
# 生成一个训练batch
#
def get_next_batch(file_name_array,text_array,batch_size=128):
    ## 图片大小 OUTPUT_SHAPE = (32, 256)
    inputs = np.zeros([batch_size, OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]])#（128,256,32）
    codes = []

    # 获取训练样本
    for i in range(batch_size):
        #从所有图片的index中，随机选取一个
        index = random.randint(0, len(file_name_array) - 1)
        #读取该张图片
        image = cv2.imread(data_dir + file_name_array[index])
        #重新缩放图片大小（256,32），那个3是什么意思
        image = cv2.resize(image, (OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]), 3)
        #cv2.cvtColor(p1,p2) 是颜色空间转换函数，p1是需要转换的图片，p2是转换成何种格式。
        #cv2.COLOR_BGR2RGB 将BGR格式转换成RGB格式
        #cv2.COLOR_BGR2GRAY 将BGR格式转换成灰度图片
        image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        # 获取训练样本数据 file_name_array, text_array = get_file_text_array()
        #取图片上对应的数字
        text = text_array[index]
        #第i个batch
        inputs[i, :] = np.transpose(image.reshape((OUTPUT_SHAPE[0], OUTPUT_SHAPE[1])))
        #每一幅图片的数字都存到列表中；作为targets使用
        codes.append(list(text))

    targets = [np.asarray(i) for i in codes]
    sparse_targets = sparse_tuple_from(targets)
    #inpus.shape=(batch_size,OUTPUT_SHAPE[1], OUTPUT_SHAPE[0])=（128,256,32）
    #一个batch所得，ctc_loss损失所需的就是长度为batch,值为 OUTPUT_SHAPE[1]一维向量
    seq_len = np.ones(inputs.shape[0]) * OUTPUT_SHAPE[1]

    return inputs, sparse_targets, seq_len

In [17]:
def get_train_model():
    inputs = tf.placeholder(tf.float32, [None, None, OUTPUT_SHAPE[0]])  # old
    targets = tf.sparse_placeholder(tf.int32)
    seq_len = tf.placeholder(tf.int32, [None])

    # 定义LSTM网络
    cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    stack = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    outputs, _ = tf.nn.dynamic_rnn(cell, inputs, seq_len, dtype=tf.float32)

    shape = tf.shape(inputs)
    batch_s, max_timesteps = shape[0], shape[1]
    outputs = tf.reshape(outputs, [-1, num_hidden])
    W = tf.Variable(tf.truncated_normal([num_hidden,
                                         num_classes],
                                        stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0., shape=[num_classes]), name="b")
    logits = tf.matmul(outputs, W) + b
    logits = tf.reshape(logits, [batch_s, -1, num_classes])

    # 转置矩阵
    logits = tf.transpose(logits, (1, 0, 2))

    return logits, inputs, targets, seq_len, W, b

In [18]:
'''
参数：

labels: 是一个 int32 类型的稀疏张量（SparseTensor）， labels.indices[i, :] == [b, t] 表示 labels.values[i] 保存着(batch b, time t)的 id，
labels.values[i] must take on values in [0, num_labels)

inputs: （常用变量 logits 表示）经过 RNN 后输出的标签预测值，是一个 3D 浮点 Tensor，
当 time_major=True(默认)时形状为：(max_time * batch_size * num_classes)，
否则形状为：batch_size * max_time * num_classes，ctc_loss will perform the softmax operation for you
 
sequence_length: 1-D int32 vector, size 为 [batch_size]，
vector 中的每个值表示序列的长度，形如 [max_time_step,…,max_time_step] ，此 sequence_length 和用在 dynamic_rnn 中的 sequence_length 是一致的，
用来表示 rnn 的哪些输出不是 pad 的.
 
preprocess_collapse_repeated: 是否需要预处理，将重复的 label 合并成一个，默认是 False
ctc_merge_repeated: 默认为 True
return :

A 1-D float Tensor, size [batch], containing the negative log probabilities，同样也需要对 ctc_loss 求均值。



tf.nn.ctc_loss(
    labels,
    inputs,
    sequence_length,
    preprocess_collapse_repeated=False,
    ctc_merge_repeated=True,
    ignore_longer_outputs_than_inputs=False,
    time_major=True
)

'''
def train():
    # 获取训练样本数据
    file_name_array, text_array = get_file_text_array()

    # 定义学习率
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                               global_step,
                                               DECAY_STEPS,
                                               LEARNING_RATE_DECAY_FACTOR,
                                               staircase=True)
    # 获取网络结构
    logits, inputs, targets, seq_len, W, b = get_train_model()

    # 设置损失函数
    #当 time_major=True(默认)时形状为：inputs:(max_time * batch_size * num_classes)，
    #return :A 1-D float Tensor, size [batch], containing the negative log probabilities，同样也需要对 ctc_loss 求均值
    loss = tf.nn.ctc_loss(labels=targets, inputs=logits, sequence_length=seq_len)#[batch_size]
    cost = tf.reduce_mean(loss)

    # 设置优化器
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    
    
    '''
    tf.nn.ctc_beam_search_decoder(
    inputs,
    sequence_length,
    beam_width=100,
    top_paths=1,
    merge_repeated=True
    )
    inputs：3-D float类型的 Tensor,大小为[max_time x batch_size x num_classes],是logits.
    sequence_length：1-D int32向量,包含序列长度,具有大小[batch_size].
    beam_width：int标量大于等于0(波束搜索波束宽度).
    top_paths：int标量大于等于0,小于等于beam_width(控制输出大小).
    merge_repeated：Boolean,默认值：True.
    '''
    
    #     元组(decoded, log_probabilities),其中：top_paths长度的列表,其中decoded[j]是SparseTensor,它包含已解码的输出：
    #log_probability：一个浮点矩阵(batch_size x top_paths),包含序列对数概率.
    decoded, log_prob = tf.nn.ctc_beam_search_decoder(logits, seq_len, merge_repeated=False)
    #编辑距离计算acc
    acc = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), targets))

    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session() as session:
        session.run(init)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=10)

        for curr_epoch in range(num_epochs):
            train_cost = 0
            train_ler = 0
            for batch in range(BATCHES):
                # 训练模型
                train_inputs, train_targets, train_seq_len = get_next_batch(file_name_array, text_array, BATCH_SIZE)
                feed = {inputs: train_inputs, targets: train_targets, seq_len: train_seq_len}
                b_loss, b_targets, b_logits, b_seq_len, b_cost, steps, _ = session.run(
                    [loss, targets, logits, seq_len, cost, global_step, optimizer], feed)

                # 评估模型
                if steps > 0 and steps % REPORT_STEPS == 0:
                    test_inputs, test_targets, test_seq_len = get_next_batch(file_name_array, text_array, BATCH_SIZE)
                    test_feed = {inputs: test_inputs,targets: test_targets,seq_len: test_seq_len}
                    dd, log_probs, accuracy = session.run([decoded[0], log_prob, acc], test_feed)
                    report_accuracy(dd, test_targets)

                    # 保存识别模型
                    save_path = saver.save(session, model_dir + "lstm_ctc_model.ctpk",global_step=steps)

                c = b_cost
                train_cost += c * BATCH_SIZE

            train_cost /= TRAIN_SIZE
            # 计算 loss
            train_inputs, train_targets, train_seq_len = get_next_batch(file_name_array, text_array, BATCH_SIZE)
            val_feed = {inputs: train_inputs,targets: train_targets,seq_len: train_seq_len}
            val_cost, val_ler, lr, steps = session.run([cost, acc, learning_rate, global_step], feed_dict=val_feed)

            log = "{} Epoch {}/{}, steps = {}, train_cost = {:.3f}, val_cost = {:.3f}"
            
            print(log.format(curr_epoch,curr_epoch + 1, num_epochs, steps, train_cost, val_cost))


In [19]:
# LSTM+CTC 文字识别能力封装
# 输入：图片
# 输出：识别结果文字
def predict(image):

# 获取网络结构
    logits, inputs, targets, seq_len, W, b = get_train_model()
    decoded, log_prob = tf.nn.ctc_beam_search_decoder(logits, seq_len, merge_repeated=False)

    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 加载模型
        saver.restore(sess, tf.train.latest_checkpoint(model_dir))
        # 图像预处理
        image = cv2.resize(image, (OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]), 3)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        pred_inputs = np.zeros([1, OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]])
        pred_inputs[0, :] = np.transpose(image.reshape((OUTPUT_SHAPE[0], OUTPUT_SHAPE[1])))
        pred_seq_len = np.ones(1) * OUTPUT_SHAPE[1]
        # 模型预测
        pred_feed = {inputs: pred_inputs,seq_len: pred_seq_len}
        dd, log_probs = sess.run([decoded[0], log_prob], pred_feed)
        # 识别结果转换
        detected_list = decode_sparse_tensor(dd)[0]
        detected_text = ''
        for d in detected_list:
            detected_text = detected_text + d

    return detected_text

In [20]:
'''
对于所有的图片，自己生成的；（batch_size,w,h）=(batch_size，256,32)=(batch_size,time_step,dim)

首先进行缩放裁剪：
OUTPUT_SHAPE = (32, 256)
cv2.resize(image, (OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]), 3)
inputs[i, :] = np.transpose(image.reshape((OUTPUT_SHAPE[0], OUTPUT_SHAPE[1])))


,最终放到模型中的输入如下

inputs = np.zeros([batch_size, OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]])#（128,256,32）
'''
train()

W0121 09:38:08.892299 139624376121088 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0121 09:38:08.894770 139624376121088 deprecation.py:323] From <ipython-input-17-9f08157a848f>:7: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0121 09:38:08.896539 139624376121088 deprecation.py:323] From <ipython-input-17-9f08157a848f>:8: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for 

0 Epoch 1/50000, steps = 10, train_cost = 450.209, val_cost = 286.500
1 Epoch 2/50000, steps = 20, train_cost = 154.562, val_cost = 50.218
2 Epoch 3/50000, steps = 30, train_cost = 30.519, val_cost = 18.955
3 Epoch 4/50000, steps = 40, train_cost = 18.595, val_cost = 17.325
4 Epoch 5/50000, steps = 50, train_cost = 17.993, val_cost = 17.545
5 Epoch 6/50000, steps = 60, train_cost = 17.698, val_cost = 16.581
6 Epoch 7/50000, steps = 70, train_cost = 17.202, val_cost = 18.658
7 Epoch 8/50000, steps = 80, train_cost = 17.680, val_cost = 17.551
8 Epoch 9/50000, steps = 90, train_cost = 17.120, val_cost = 16.674
len(original_list) 64 len(detected_list) 37  test and detect length desn't match
9 Epoch 10/50000, steps = 100, train_cost = 16.758, val_cost = 17.370
10 Epoch 11/50000, steps = 110, train_cost = 16.360, val_cost = 16.876
11 Epoch 12/50000, steps = 120, train_cost = 16.474, val_cost = 16.244
12 Epoch 13/50000, steps = 130, train_cost = 16.744, val_cost = 16.858
13 Epoch 14/50000, st

69 Epoch 70/50000, steps = 700, train_cost = 3.361, val_cost = 3.180
70 Epoch 71/50000, steps = 710, train_cost = 3.125, val_cost = 3.312
71 Epoch 72/50000, steps = 720, train_cost = 3.017, val_cost = 2.594
72 Epoch 73/50000, steps = 730, train_cost = 2.892, val_cost = 2.944
73 Epoch 74/50000, steps = 740, train_cost = 2.734, val_cost = 2.691
74 Epoch 75/50000, steps = 750, train_cost = 2.712, val_cost = 2.911
75 Epoch 76/50000, steps = 760, train_cost = 2.613, val_cost = 2.405
76 Epoch 77/50000, steps = 770, train_cost = 2.578, val_cost = 2.366
77 Epoch 78/50000, steps = 780, train_cost = 2.414, val_cost = 2.071
78 Epoch 79/50000, steps = 790, train_cost = 2.241, val_cost = 2.032
T/F: original(length) <-------> detectcted(length)
True ['4'] ( 1 ) <------->  ['4'] ( 1 )
False ['4', '6', '7', '3', '0', '2', '8', '8', '3', '4'] ( 10 ) <------->  ['4', '6', '3', '0', '2', '8', '8', '3', '4'] ( 9 )
False ['0', '2', '7', '7', '6', '8', '4', '6', '3'] ( 9 ) <------->  ['0', '0', '2', '7', '7

89 Epoch 90/50000, steps = 900, train_cost = 1.625, val_cost = 1.721
90 Epoch 91/50000, steps = 910, train_cost = 1.495, val_cost = 1.605
91 Epoch 92/50000, steps = 920, train_cost = 1.506, val_cost = 1.722
92 Epoch 93/50000, steps = 930, train_cost = 1.520, val_cost = 1.341
93 Epoch 94/50000, steps = 940, train_cost = 1.374, val_cost = 1.317
94 Epoch 95/50000, steps = 950, train_cost = 1.409, val_cost = 1.227
95 Epoch 96/50000, steps = 960, train_cost = 1.342, val_cost = 1.373
96 Epoch 97/50000, steps = 970, train_cost = 1.303, val_cost = 1.183
97 Epoch 98/50000, steps = 980, train_cost = 1.302, val_cost = 1.230
98 Epoch 99/50000, steps = 990, train_cost = 1.209, val_cost = 1.227
T/F: original(length) <-------> detectcted(length)
True ['1', '5', '2', '8', '4', '9', '9', '3'] ( 8 ) <------->  ['1', '5', '2', '8', '4', '9', '9', '3'] ( 8 )
True ['1', '0', '9'] ( 3 ) <------->  ['1', '0', '9'] ( 3 )
True ['2', '0', '3', '4', '2'] ( 5 ) <------->  ['2', '0', '3', '4', '2'] ( 5 )
False ['7

W0121 09:47:51.214343 139624376121088 deprecation.py:323] From /home/amis/.local/lib/python3.5/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


...

119 Epoch 120/50000, steps = 1200, train_cost = 0.785, val_cost = 0.698
120 Epoch 121/50000, steps = 1210, train_cost = 0.722, val_cost = 0.692
121 Epoch 122/50000, steps = 1220, train_cost = 0.720, val_cost = 0.816
122 Epoch 123/50000, steps = 1230, train_cost = 0.729, val_cost = 0.722
123 Epoch 124/50000, steps = 1240, train_cost = 0.693, val_cost = 0.764
124 Epoch 125/50000, steps = 1250, train_cost = 0.657, val_cost = 0.689
125 Epoch 126/50000, steps = 1260, train_cost = 0.667, val_cost = 0.744
126 Epoch 127/50000, steps = 1270, train_cost = 0.652, val_cost = 0.644
127 Epoch 128/50000, steps = 1280, train_cost = 0.647, val_cost = 0.743
128 Epoch 129/50000, steps = 1290, train_cost = 0.584, val_cost = 0.554
T/F: original(length) <-------> detectcted(length)
True ['5'] ( 1 ) <------->  ['5'] ( 1 )
True ['2'] ( 1 ) <------->  ['2'] ( 1 )
True ['7', '5', '6', '6', '6'] ( 5 ) <------->  ['7', '5', '6', '6', '6'] ( 5 )
True ['9'] ( 1 ) <------->  ['9'] ( 1 )
True ['9', '1', '2', '4', '8'

139 Epoch 140/50000, steps = 1400, train_cost = 0.527, val_cost = 0.384
140 Epoch 141/50000, steps = 1410, train_cost = 0.505, val_cost = 0.482
141 Epoch 142/50000, steps = 1420, train_cost = 0.515, val_cost = 0.478
142 Epoch 143/50000, steps = 1430, train_cost = 0.489, val_cost = 0.437
143 Epoch 144/50000, steps = 1440, train_cost = 0.520, val_cost = 0.454
144 Epoch 145/50000, steps = 1450, train_cost = 0.473, val_cost = 0.429
145 Epoch 146/50000, steps = 1460, train_cost = 0.480, val_cost = 0.417
146 Epoch 147/50000, steps = 1470, train_cost = 0.461, val_cost = 0.495
147 Epoch 148/50000, steps = 1480, train_cost = 0.479, val_cost = 0.429
148 Epoch 149/50000, steps = 1490, train_cost = 0.430, val_cost = 0.548
len(original_list) 64 len(detected_list) 63  test and detect length desn't match
149 Epoch 150/50000, steps = 1500, train_cost = 0.440, val_cost = 0.396
150 Epoch 151/50000, steps = 1510, train_cost = 0.413, val_cost = 0.429
151 Epoch 152/50000, steps = 1520, train_cost = 0.437, 

169 Epoch 170/50000, steps = 1700, train_cost = 0.318, val_cost = 0.294
170 Epoch 171/50000, steps = 1710, train_cost = 0.337, val_cost = 0.348
171 Epoch 172/50000, steps = 1720, train_cost = 0.328, val_cost = 0.365
172 Epoch 173/50000, steps = 1730, train_cost = 0.339, val_cost = 0.328
173 Epoch 174/50000, steps = 1740, train_cost = 0.307, val_cost = 0.263
174 Epoch 175/50000, steps = 1750, train_cost = 0.306, val_cost = 0.249
175 Epoch 176/50000, steps = 1760, train_cost = 0.324, val_cost = 0.307
176 Epoch 177/50000, steps = 1770, train_cost = 0.315, val_cost = 0.301
177 Epoch 178/50000, steps = 1780, train_cost = 0.303, val_cost = 0.357
178 Epoch 179/50000, steps = 1790, train_cost = 0.307, val_cost = 0.309
T/F: original(length) <-------> detectcted(length)
False ['5'] ( 1 ) <------->  [] ( 0 )
True ['2', '8'] ( 2 ) <------->  ['2', '8'] ( 2 )
True ['3', '3', '0', '6', '0', '7', '2', '9'] ( 8 ) <------->  ['3', '3', '0', '6', '0', '7', '2', '9'] ( 8 )
True ['8', '3', '1', '4', '8', 

189 Epoch 190/50000, steps = 1900, train_cost = 0.287, val_cost = 0.314
190 Epoch 191/50000, steps = 1910, train_cost = 0.282, val_cost = 0.247
191 Epoch 192/50000, steps = 1920, train_cost = 0.259, val_cost = 0.230
192 Epoch 193/50000, steps = 1930, train_cost = 0.253, val_cost = 0.272
193 Epoch 194/50000, steps = 1940, train_cost = 0.257, val_cost = 0.279
194 Epoch 195/50000, steps = 1950, train_cost = 0.250, val_cost = 0.248
195 Epoch 196/50000, steps = 1960, train_cost = 0.254, val_cost = 0.251
196 Epoch 197/50000, steps = 1970, train_cost = 0.243, val_cost = 0.242
197 Epoch 198/50000, steps = 1980, train_cost = 0.226, val_cost = 0.219
198 Epoch 199/50000, steps = 1990, train_cost = 0.230, val_cost = 0.289
T/F: original(length) <-------> detectcted(length)
True ['2', '6', '4', '7', '8'] ( 5 ) <------->  ['2', '6', '4', '7', '8'] ( 5 )
True ['0', '1', '2', '1', '2', '1', '2', '6'] ( 8 ) <------->  ['0', '1', '2', '1', '2', '1', '2', '6'] ( 8 )
True ['2', '0', '2', '1', '8', '8', '2'

KeyboardInterrupt: 